# LCEL Tutorial

See blog: https://theepsilon.substack.com/p/brief-notes-on-lcel

In [ ]:
!pip install --upgrade --quiet  langchain docarray tiktoken langchain-core langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#set up key in secrets.

from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

## Starter Example

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """ tell me something interesting about {name}"""
    )
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"name": "Mother Theresa"})

Mother Teresa was born in North Macedonia in 1910, and baptized as Agnes Gonxha Bojaxhiu. She moved to Ireland at the age of 18 to join the Sisters of Loreto and then to India a year later, where she lived most of her life. She founded the Missionaries of Charity in 1950, which is a Roman Catholic religious congregation that had over 4,500 sisters and was active in 133 countries as of 2012. The congregation manages homes for people dying of HIV/AIDS, leprosy, and tuberculosis. It also runs soup kitchens, dispensaries, mobile clinics, children's and family counselling programs, orphanages, and schools. Mother Teresa was canonized as a saint in the Roman Catholic Church in 2016, just 19 years after her death.

`Stream`

In [ ]:
for chunk in chain.stream({"name": "Mother Theresa"}):
    print(chunk, end="", flush=True)

`Batch`

In [ ]:
chain.batch(["Mother Theresa", "Barrack Obama"])

['Mother Teresa, known in the Catholic Church as Saint Teresa of Calcutta, was actually born with the name Anjezë Gonxhe Bojaxhiu in 1910 in Skopje, which was then part of the Ottoman Empire and is now the capital of North Macedonia. Despite being globally recognized for her humanitarian work in India, she was actually of Albanian ethnicity. She moved to Ireland at age 18 to join the Sisters of Loreto and then to India a year later, where she lived most of her life.',
 'Barack Obama is known for being the 44th President of the United States, but an interesting fact many people might not know is that he won a Grammy Award. Before his presidency, he won the Grammy for Best Spoken Word Album in 2006 for his audiobook, "Dreams From My Father." He won the same award again in 2008 for his audiobook, "The Audacity of Hope."']

`Async`

In [ ]:
await chain.ainvoke("Mother Theresa")

`Async Batch`

In [ ]:
await chain.abatch(["Mother Theresa", "Barrack Obama"])

*All together.*

In [ ]:
chain.invoke({"name": "Mother Theresa"})
for chunk in chain.stream({"name": "Mother Theresa"}):
    print(chunk, end="", flush=True)
await chain.ainvoke("Mother Theresa")
chain.batch(["Mother Theresa", "Barrack Obama"])
await chain.abatch(["Mother Theresa", "Barrack Obama"])

## RAG Example

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["Bifarin is a post-doc at Georgia Tech",
     "Edward III was the King of England from 1327 to 1377"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("who was Edward III?")

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Edward III was the King of England from 1327 to 1377.'